In [7]:
import dgl
import torch
import numpy as np
import torch.nn as nn
import pickle
import torch.nn.functional as F

n=1536 #13824 #1534
drop_prob=0
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_heads=1
control_str='S'

def getmodel():
    g = pickle.load(open("UpdatedGraph_Neighbour10_Coarsen3", 'rb'))
    adjMatrix=g.adj(scipy_fmt='coo')
    aa=adjMatrix.todense()
    aa=torch.from_numpy(aa).float()

    def model(x):
        return torch.matmul(x[:,:,0,:],aa)
    return model,g

model,g=getmodel()
# model = STGCN_WAVE([1,1], 1, 1, n, g, drop_prob, device, num_heads, control_str).to(device)
model = UnetGraphSAGE(g,1,256, 1,2)


# x=torch.rand(1,1,1,n)
x=torch.rand(n,1)

x.requires_grad_()
y=model(x)
# y[0,0,0,100].backward()
y[100,0].backward()

dx=x.grad
(dx!=0).sum()


tensor(694)

In [1]:
from dgl.nn import SAGEConv
import torch.nn as nn

class UnetGraphSAGE(nn.Module):
    def __init__(self, g, in_feats, h_feats,out_feat,num_step):
        super(UnetGraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, int(h_feats/2), 'mean')
        self.conv3 = SAGEConv(int(h_feats/2), int(h_feats/4), 'mean')
        self.conv4 = SAGEConv(int(h_feats/4), int(h_feats/4), 'mean')
        self.conv5 = SAGEConv(int(h_feats/2), int(h_feats/2), 'mean')
        self.conv6 = SAGEConv(h_feats, out_feat, 'mean')
        self.g=g
        self.num_step=num_step
    def forward(self, in_feat):

        for _ in range(self.num_step):
            h = self.conv1(g, in_feat)
            h = F.relu(h)
            h = self.conv2(g, h)
            h = F.relu(h)
            h = self.conv3(g, h)
            h = F.relu(h)
            tuple = (self.conv4(self.g, h),h)
            h = torch.cat(tuple,dim=1)
            h = F.relu(h)
            tuple = (self.conv5(self.g, h),h)
            h = torch.cat(tuple,dim=1)
            h = F.relu(h)
            h = self.conv6(self.g, h)
            in_feat=h
        return h

/home/ebrahimn/miniconda3/envs/dlwp2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
